In [ ]:
import os
os.chdir("..")

In [ ]:
import h5py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sb
from TravelAndMutate.datamanager import getHeteroAttributes

In [ ]:
filename = "alphaUK"
plotdir = f"analysis/{filename}/fitness_evolution/"
if not os.path.isdir(plotdir):
	os.makedirs(plotdir)
file = h5py.File(f"analysis/{filename}.h5")

attrs = getHeteroAttributes(file)
attrs

In [ ]:
attrs["R0"] = attrs["betas"] / 0.2
attrs

In [ ]:
labels = {
	"fitness_p" : r"$P\left(\Delta\varphi^+\right)$",
	"fitness_delta+" : r"$\Delta\varphi^+$",
	"fitness_delta-" : r"$\Delta\varphi^-$",
	"mutation_rate" : r"mean($\eta$)",
	"betas" : r"$\beta$",
	"epsilons" : r"$\varepsilon$",
	"mus" : r"$\mu$"
}

In [ ]:
def plot(file, fixed_params, alongrows, alongcolumns):
	mask = np.ones_like(attrs.index, dtype=bool)
	for key,val in fixed_params.items():
		mask = np.logical_and(mask, attrs[key] == val)
	data = attrs.loc[mask].copy().sort_values([alongcolumns,alongrows], ascending=[False,True])
	groupnames = data.index
	nrows = data[alongcolumns].unique().shape[0]
	ncols = data[alongrows].unique().shape[0]
	fig, axs = plt.subplots(nrows, ncols, figsize=(4*ncols,3*nrows))
	axs = axs.flatten()
	for i,name in enumerate(groupnames):
		if "fitness_evolution" in file[name]:
			to_plot = pd.DataFrame.from_records(file[f"{name}/fitness_evolution"][:])
			sb.histplot(data=to_plot, x="t", y="phi", bins=50, cmap="Blues", ax=axs[i])
		label = [
			r"$P\left(\Delta\varphi^+\right)$"+f" = {round(attrs.loc[name]['fitness_p'], 2)}",
			r"mean($\eta$)"+f" = {attrs.loc[name]['mutation_rate']}"
		]
		axs[i].text(0.05, 0.2, "\n".join(label), transform=axs[i].transAxes)
		axs[i].set_ylabel("Fitness")
		axs[i].set_xlabel("t")
		axs[i].set_xlim(left=0)
	title = "\n".join([f"{labels[key]} = {str(val)}" for key,val in fixed_params.items()])
	fig.suptitle(title, y=1)
	fig.tight_layout()
	filename = plotdir + "-".join(sorted(data.index.tolist())) + ".png"
	return filename, fig

In [ ]:
for deltap in attrs["fitness_delta+"].unique():
	for deltam in attrs["fitness_delta-"].unique():
		for beta in attrs["betas"].unique():
			fixed = {"fitness_delta-":deltam, "fitness_delta+":deltap, "betas":beta}
			filename, fig = plot(file, fixed, "mutation_rate", "fitness_p")
			fig.savefig(filename)
			plt.close(fig)

In [ ]:
file.close()